In [ ]:
# Overlap Weights and Covariate Balance

# This notebook calculates overlap weights and merges cohort and lab data for evaluating covariate balance study of semaglutide and tirzepatide in HFpEF.

# Required inputs:
## `cohort_all_followup.csv`
## `cohort_all_followup_key.csv`
## `labs_all_followup.csv`
## `labs_all_followup_key.csv`

# Output:
## A merged DataFrame with overlap weights

In [1]:
import pandas as pd
import numpy as np

# Load datasets
cohort_all_followup = pd.read_csv("path/to/cohort_file.csv")
cohort_all_followup_key = pd.read_csv("path/to/cohort_file.csv")

labs_all_followup = pd.read_csv("path/to/cohort_file.csv")
labs_all_followup_key = pd.read_csv("path/to/cohort_file.csv")

# Create and apply a column renaming mapping for cohort_all_followup
cohort_mapping = dict(zip(cohort_all_followup_key["Column Name"], cohort_all_followup_key["Description"]))
cohort_mapping.pop('PID', None)  # Ensure 'PID' remains unchanged
trimmed_cohort_mapping = {col: desc.split(" - ")[-1] for col, desc in cohort_mapping.items()}
cohort_all_followup.rename(columns=trimmed_cohort_mapping, inplace=True)

# Create and apply column renaming mapping for lab columns explicitly
labs_mapping = dict(zip(labs_all_followup_key["Column Name"], labs_all_followup_key["Description"]))
selected_lab_cols = ['RUN1_ENTRY_COVARIATE_1', 'RUN1_ENTRY_COVARIATE_2', 'RUN1_ENTRY_COVARIATE_3', 'RUN1_ENTRY_COVARIATE_4', 'RUN1_ENTRY_COVARIATE_5', 'RUN1_ENTRY_COVARIATE_6', 'RUN1_ENTRY_COVARIATE_7', 'RUN1_ENTRY_COVARIATE_8', 'RUN1_ENTRY_COVARIATE_9',]

lab_subset = labs_all_followup[['PID'] + selected_lab_cols].rename(
    columns={col: labs_mapping[col].split(" - ")[-1] for col in selected_lab_cols}
)

# Merge cohort with lab data based on 'PID'
merged_data = pd.merge(cohort_all_followup, lab_subset, on='PID', how='left')

# Calculate overlap weights
merged_data['overlap_weight'] = merged_data.apply(
    lambda row: 1 - row['Propensity Score'] if row['Exposure Group'] == 'E' else row['Propensity Score'], axis=1
)

# Inspect the final merged dataset
merged_data

,PID,Source Patient ID,Exposure Group,Age,Gender (male),ACEi / ARB,Acute kidney injury,Anemia,Anti-arrhythmics,Anti-depressants,...,BMI (Truncated),BNP (Truncated),proBNP (Truncated),HbA1c (Truncated),Glucose (Truncated),eGFR (Truncated),Creatinine (Truncated),Systolic blood pressure (Truncated),Heart rate (Truncated),overlap_weight
0,1863,33170810471,E,77,False,True,False,False,False,True,...,46.38,NaN,1377.0,11.6,87.000000,26.0,1.98,160.0,53.0,0.038780
1,4206,33285703803,E,71,True,True,False,False,False,False,...,37.05,NaN,NaN,8.4,187.500000,77.0,1.04,150.0,60.0,0.206714
2,12017,33250396649,E,67,False,True,False,False,False,True,...,NaN,NaN,NaN,NaN,141.000000,79.0,0.82,NaN,NaN,0.051516
3,20058,33016272826,E,79,False,True,False,False,False,False,...,43.00,NaN,NaN,NaN,159.333333,NaN,0.99,126.0,NaN,0.462813
4,29514,33047614501,R,50,True,True,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.869567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11347,4290672336441,33039421417,R,77,True,True,False,False,False,False,...,NaN,NaN,NaN,9.0,177.000000,NaN,1.30,NaN,NaN,0.048136
11348,4290672349105,33160694333,E,82,False,True,True,True,False,False,...,NaN,NaN,NaN,9.3,154.000000,NaN,1.41,NaN,NaN,0.769347
11349,4290672356231,33055623006,R,75,False,True,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.091140
11350,4290672362646,33182884796,R,67,True,True,True,False,False,False,...,NaN,NaN,NaN,9.3,372.000000,NaN,1.41,NaN,NaN,0.203392


In [2]:
# Count the number of occurrences for each Exposure Group (R and E)
exposure_group_counts = merged_data['Exposure Group'].value_counts()
print(exposure_group_counts)

Exposure Group
E    8022
R    3330
Name: count, dtype: int64


In [3]:
 # Separate exposure groups
exposed = merged_data[merged_data['Exposure Group'] == 'E'].copy()
referent = merged_data[merged_data['Exposure Group'] == 'R'].copy()

# Define ESS function
def compute_ess(weights):
    return (np.sum(weights) ** 2) / np.sum(weights ** 2)

# Extract raw overlap weights
weights_exp_raw = exposed['overlap_weight']
weights_ref_raw = referent['overlap_weight']

# Compute ESS for each group
ess_exp = compute_ess(weights_exp_raw)
ess_ref = compute_ess(weights_ref_raw)

# Print summary
print(f"Raw count exposed: {len(exposed):,}")
print(f"Raw count referent: {len(referent):,}")
print(f"ESS exposed: {ess_exp:,.1f}")
print(f"ESS referent: {ess_ref:,.1f}")
print(f"Reduction in sample size (exposed): {100 * (1 - ess_exp / len(exposed)):.1f}%")
print(f"Reduction in sample size (referent): {100 * (1 - ess_ref / len(referent)):.1f}%")

Raw count exposed: 8,022
Raw count referent: 3,330
ESS exposed: 3,783.8
ESS referent: 2,085.8
Reduction in sample size (exposed): 52.8%
Reduction in sample size (referent): 37.4%


In [4]:
# Total sum of weights per group
total_weights_per_group = merged_data.groupby('Exposure Group')['overlap_weight'].sum()
print(total_weights_per_group)

Exposure Group
E    1341.598969
R    1341.598957
Name: overlap_weight, dtype: float64


# Patient characteristics (unweighted)

In [8]:
import pandas as pd
import numpy as np
from decimal import Decimal, ROUND_HALF_UP

# Helper rounding functions
def round_prop(val):
    return Decimal(val * 100).quantize(Decimal('1.0'), rounding=ROUND_HALF_UP)

def round_sd(val):
    return round(val, 2)

def round_int(val):
    return int(round(val))

# SMD helper
def calc_smd(exp, ref):
    mean_exp, mean_ref = np.mean(exp), np.mean(ref)
    pooled_sd = np.sqrt((np.var(exp, ddof=1) + np.var(ref, ddof=1)) / 2)
    return (mean_exp - mean_ref) / pooled_sd if pooled_sd else np.nan

# Separate groups
exposed = merged_data[merged_data['Exposure Group'] == 'E']
referent = merged_data[merged_data['Exposure Group'] == 'R']
n_exp = len(exposed)
n_ref = len(referent)

numeric_cols = merged_data.select_dtypes(include=['float64', 'int64']).columns.drop(['PID', 'Follow Up Time', 'Propensity Score', 'overlap_weight'])
bool_cols = merged_data.select_dtypes(include=['bool']).columns

categorical_split_vars = [
    "Number of hospitalizations (0, 1, 2 or more)",
    "Number of heart failure hospitalizations (0, 1, 2 or more)",
    "Weight Categorization Score",
    "Entry Classification",
    "Region / State",
    "Race (Recategorized)"
]

bins_labels = {
    "Number of hospitalizations (0, 1, 2 or more)": ([-1, 0, 1, float('inf')], ["0", "1", "2 or more"]),
    "Number of heart failure hospitalizations (0, 1, 2 or more)": ([-1, 0, 1, float('inf')], ["0", "1", "2 or more"]),
    "Weight Categorization Score": ([-1, 0, 1, 2, 3], [
        "Class 1 Obesity (30.0-34.9)",
        "Class 2 Obesity (35.0-39.9)",
        "Class 3 Obesity (40.0 and above)",
        "Unspecified Obesity"
    ]),
    "Entry Classification": ([2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024], ["2018", "2019", "2020", "2021", "2022", "2023", "2024"]),
    "Region / State": ([-1, 0, 1, 2, 3, 4], ["Northest", "Midwest / North central", "South", "West", "Missing"]),
    "Race (Recategorized)": ([-1, 0, 1, 2, 3], ["White", "Other", "Black", "Unknown / Missing"])
}

unweighted_results = []

# Binary categorical variables
for col in bool_cols:
    exp_count = exposed[col].sum()
    ref_count = referent[col].sum()
    exp_pct = exp_count / n_exp
    ref_pct = ref_count / n_ref
    smd = calc_smd(exposed[col].astype(int), referent[col].astype(int))

    unweighted_results.append({
        'Variable': col,
        'Type': 'Categorical',
        'Exposure': f"{round_int(exp_count)} ({round_prop(exp_pct)}%)",
        'Referent': f"{round_int(ref_count)} ({round_prop(ref_pct)}%)",
        'SMD': f"{smd:.3f}"
    })

# Continuous variables
for col in numeric_cols:
    if col in categorical_split_vars:
        continue

    mean_exp, std_exp = exposed[col].mean(), exposed[col].std()
    median_exp, q1_exp, q3_exp = exposed[col].median(), exposed[col].quantile(0.25), exposed[col].quantile(0.75)

    mean_ref, std_ref = referent[col].mean(), referent[col].std()
    median_ref, q1_ref, q3_ref = referent[col].median(), referent[col].quantile(0.25), referent[col].quantile(0.75)

    smd = calc_smd(exposed[col].dropna(), referent[col].dropna())

    unweighted_results.append({
        'Variable': col,
        'Type': 'Continuous',
        'Exposure': f"{mean_exp:.2f} ± {round_sd(std_exp):.2f} [{round_int(median_exp)}, {round_int(q1_exp)}-{round_int(q3_exp)}]",
        'Referent': f"{mean_ref:.2f} ± {round_sd(std_ref):.2f} [{round_int(median_ref)}, {round_int(q1_ref)}-{round_int(q3_ref)}]",
        'SMD': f"{smd:.3f}"
    })

# Expanded categorical variables
for col in categorical_split_vars:
    bins, labels = bins_labels[col]
    exposed_cat = pd.cut(exposed[col], bins=bins, labels=labels)
    referent_cat = pd.cut(referent[col], bins=bins, labels=labels)

    for level in labels:
        mask_exp = (exposed_cat == level)
        mask_ref = (referent_cat == level)

        exp_count = mask_exp.sum()
        ref_count = mask_ref.sum()
        exp_pct = exp_count / len(exposed_cat)
        ref_pct = ref_count / len(referent_cat)

        smd = calc_smd(mask_exp.astype(int), mask_ref.astype(int))

        unweighted_results.append({
            'Variable': f"{col} = {level}",
            'Type': 'Categorical',
            'Exposure': f"{round_int(exp_count)} ({round_prop(exp_pct)}%)",
            'Referent': f"{round_int(ref_count)} ({round_prop(ref_pct)}%)",
            'SMD': f"{smd:.3f}"
        })

# Label columns with group size
exp_label = f"Exposure (n = {n_exp})"
ref_label = f"Referent (n = {n_ref})"

final_unweighted_table = (
    pd.DataFrame(unweighted_results)
    .sort_values(by=["Type", "Variable"])
    .reset_index(drop=True)
    .rename(columns={
        "Exposure": exp_label,
        "Referent": ref_label
    })
)

final_unweighted_table

,Variable,Type,Exposure (n = 8022),Referent (n = 3330),SMD
0,"Colonoscopy, Sigmoidoscopy",Categorical,726 (9.1%),239 (7.2%),0.069
1,ACEi / ARB,Categorical,5867 (73.1%),2546 (76.5%),-0.077
2,ARNI,Categorical,582 (7.3%),126 (3.8%),0.152
3,Acute kidney injury,Categorical,1482 (18.5%),677 (20.3%),-0.047
4,Anemia,Categorical,2399 (29.9%),1150 (34.5%),-0.099
...,...,...,...,...,...
155,Unique brand medications,Continuous,"20.22 ± 7.73 [19, 15-25]","19.43 ± 7.37 [18, 14-24]",0.106
156,Unique generic medications,Continuous,"19.76 ± 7.39 [19, 14-24]","19.01 ± 7.05 [18, 14-23]",0.104
157,Urine tests,Continuous,"1.34 ± 1.94 [1, 0-2]","1.46 ± 2.02 [1, 0-2]",-0.065
158,eGFR (Truncated),Continuous,"64.48 ± 23.53 [64, 46-83]","60.00 ± 22.37 [59, 44-76]",0.195


# Patient characteristics (weighted)

In [9]:
import pandas as pd
import numpy as np
from decimal import Decimal, ROUND_HALF_UP

# Rounding helpers
def round_prop(val):
    return Decimal(val * 100).quantize(Decimal('1.0'), rounding=ROUND_HALF_UP)

def round_sd(val):
    return round(val, 2)

def round_int(val):
    return int(round(val))

# Weighted helper functions
def weighted_mean_std(data, weights):
    mean = np.average(data, weights=weights)
    variance = np.average((data - mean)**2, weights=weights)
    return mean, np.sqrt(variance)

def weighted_median_iqr(data, weights):
    sorted_indices = np.argsort(data)
    sorted_data = data[sorted_indices]
    sorted_weights = weights[sorted_indices]
    cum_weights = np.cumsum(sorted_weights)
    total_weight = cum_weights[-1]

    median = np.interp(total_weight / 2, cum_weights, sorted_data)
    q1 = np.interp(total_weight / 4, cum_weights, sorted_data)
    q3 = np.interp(3 * total_weight / 4, cum_weights, sorted_data)

    return median, q1, q3

def weighted_smd(exp_data, exp_weights, ref_data, ref_weights):
    mean_exp, _ = weighted_mean_std(exp_data, exp_weights)
    mean_ref, _ = weighted_mean_std(ref_data, ref_weights)

    pooled_var = (
        np.average((exp_data - mean_exp) ** 2, weights=exp_weights) * np.sum(exp_weights) +
        np.average((ref_data - mean_ref) ** 2, weights=ref_weights) * np.sum(ref_weights)
    ) / (np.sum(exp_weights) + np.sum(ref_weights))

    pooled_sd = np.sqrt(pooled_var)
    return (mean_exp - mean_ref) / pooled_sd if pooled_sd else np.nan

# Separate groups
exposed = merged_data[merged_data['Exposure Group'] == 'E'].copy()
referent = merged_data[merged_data['Exposure Group'] == 'R'].copy()

# Use raw overlap weights (TSW) directly
weights_exp_raw = exposed['overlap_weight'].values
weights_ref_raw = referent['overlap_weight'].values

# Total sum of weights for each group
tsw_exp = np.sum(weights_exp_raw)
tsw_ref = np.sum(weights_ref_raw)

# Assign TSW‐based weights
exposed['tsw_weight'] = weights_exp_raw
referent['tsw_weight'] = weights_ref_raw

# Identify columns
numeric_cols = merged_data\
    .select_dtypes(include=['float64', 'int64'])\
    .columns.drop(['PID', 'Follow Up Time', 'Propensity Score', 'overlap_weight'])
bool_cols = merged_data.select_dtypes(include=['bool']).columns

categorical_split_vars = [
    "Number of hospitalizations (0, 1, 2 or more)",
    "Number of heart failure hospitalizations (0, 1, 2 or more)",
    "Weight Categorization Score",
    "Entry Classification",
    "Region / State",
    "Race (Recategorized)"
]

bins_labels = {
    "Number of hospitalizations (0, 1, 2 or more)": ([-1, 0, 1, float('inf')], ["0", "1", "2 or more"]),
    "Number of heart failure hospitalizations (0, 1, 2 or more)": ([-1, 0, 1, float('inf')], ["0", "1", "2 or more"]),
    "Weight Categorization Score": ([-1, 0, 1, 2, 3], [
        "Class 1 Obesity (30.0-34.9)",
        "Class 2 Obesity (35.0-39.9)",
        "Class 3 Obesity (40.0 and above)",
        "Unspecified Obesity"
    ]),
    "Entry Classification": ([2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024], ["2018", "2019", "2020", "2021", "2022", "2023", "2024"]),
    "Region / State": ([-1, 0, 1, 2, 3, 4], ["Northest", "Midwest / North central", "South", "West", "Missing"]),
    "Race (Recategorized)": ([-1, 0, 1, 2, 3], ["White", "Other", "Black", "Unknown / Missing"])
}

weighted_results = []

# Binary categorical variables
for col in bool_cols:
    exp_count = np.sum(exposed[col] * exposed['tsw_weight'])
    exp_total = tsw_exp
    ref_count = np.sum(referent[col] * referent['tsw_weight'])
    ref_total = tsw_ref
    smd = weighted_smd(
        exposed[col].astype(int), exposed['tsw_weight'],
        referent[col].astype(int), referent['tsw_weight']
    )

    weighted_results.append({
        'Variable': col,
        'Type': 'Categorical',
        'Weighted Exposure': f"{round_int(exp_count)} ({round_prop(exp_count / exp_total)}%)",
        'Weighted Referent': f"{round_int(ref_count)} ({round_prop(ref_count / ref_total)}%)",
        'Weighted SMD': f"{smd:.3f}"
    })

# Continuous variables
for col in numeric_cols:
    if col in categorical_split_vars:
        continue

    exp_data = exposed[col].dropna().values
    exp_w = exposed.loc[exposed[col].notna(), 'tsw_weight'].values
    ref_data = referent[col].dropna().values
    ref_w = referent.loc[referent[col].notna(), 'tsw_weight'].values

    exp_mean, exp_std = weighted_mean_std(exp_data, exp_w)
    ref_mean, ref_std = weighted_mean_std(ref_data, ref_w)

    exp_median, exp_q1, exp_q3 = weighted_median_iqr(exp_data, exp_w)
    ref_median, ref_q1, ref_q3 = weighted_median_iqr(ref_data, ref_w)

    smd = weighted_smd(exp_data, exp_w, ref_data, ref_w)

    weighted_results.append({
        'Variable': col,
        'Type': 'Continuous',
        'Weighted Exposure': f"{exp_mean:.2f} ± {round_sd(exp_std):.2f} [{round_int(exp_median)}, {round_int(exp_q1)}-{round_int(exp_q3)}]",
        'Weighted Referent': f"{ref_mean:.2f} ± {round_sd(ref_std):.2f} [{round_int(ref_median)}, {round_int(ref_q1)}-{round_int(ref_q3)}]",
        'Weighted SMD': f"{smd:.3f}"
    })

# Expanded categorical variables
for col in categorical_split_vars:
    bins, labels = bins_labels[col]
    exposed_cat = pd.cut(exposed[col], bins=bins, labels=labels)
    referent_cat = pd.cut(referent[col], bins=bins, labels=labels)

    for level in labels:
        mask_exp = (exposed_cat == level)
        mask_ref = (referent_cat == level)

        exp_count = np.sum(exposed.loc[mask_exp, 'tsw_weight'])
        ref_count = np.sum(referent.loc[mask_ref, 'tsw_weight'])
        exp_pct = exp_count / tsw_exp
        ref_pct = ref_count / tsw_ref

        smd = weighted_smd(mask_exp.astype(int), exposed['tsw_weight'],
                           mask_ref.astype(int), referent['tsw_weight'])

        weighted_results.append({
            'Variable': f"{col} = {level}",
            'Type': 'Categorical',
            'Weighted Exposure': f"{round_int(exp_count)} ({round_prop(exp_pct)}%)",
            'Weighted Referent': f"{round_int(ref_count)} ({round_prop(ref_pct)}%)",
            'Weighted SMD': f"{smd:.3f}"
        })

# Final table (now labeled by TSW)
tsw_label_exp = f"Weighted Exposure (n = {int(round(tsw_exp))})"
tsw_label_ref = f"Weighted Referent (n = {int(round(tsw_ref))})"

final_weighted_table = (
    pd.DataFrame(weighted_results)
      .sort_values(by=["Type", "Variable"])
      .reset_index(drop=True)
      .rename(columns={
          "Weighted Exposure": tsw_label_exp,
          "Weighted Referent": tsw_label_ref
      })
)

final_weighted_table

,Variable,Type,Weighted Exposure (n = 1342),Weighted Referent (n = 1342),Weighted SMD
0,"Colonoscopy, Sigmoidoscopy",Categorical,101 (7.5%),101 (7.5%),-0.000
1,ACEi / ARB,Categorical,1010 (75.3%),1010 (75.3%),0.000
2,ARNI,Categorical,69 (5.1%),69 (5.1%),-0.000
3,Acute kidney injury,Categorical,265 (19.7%),265 (19.7%),-0.000
4,Anemia,Categorical,448 (33.4%),448 (33.4%),-0.000
...,...,...,...,...,...
155,Unique brand medications,Continuous,"19.87 ± 7.60 [19, 14-24]","19.87 ± 7.59 [19, 14-24]",0.001
156,Unique generic medications,Continuous,"19.41 ± 7.24 [18, 14-24]","19.41 ± 7.25 [18, 14-24]",-0.000
157,Urine tests,Continuous,"1.41 ± 1.98 [1, 0-2]","1.41 ± 2.00 [1, 0-2]",-0.000
158,eGFR (Truncated),Continuous,"62.43 ± 22.22 [61, 46-80]","60.92 ± 22.67 [59, 45-78]",0.068


In [10]:
# Identify the actual weighted column names (with sample size info)
weighted_exp_col = [col for col in final_weighted_table.columns if col.startswith("Weighted Exposure")][0]
weighted_ref_col = [col for col in final_weighted_table.columns if col.startswith("Weighted Referent")][0]

# Identify unweighted column names with sample size info
unweighted_exp_col = [col for col in final_unweighted_table.columns if col.startswith("Exposure")][0]
unweighted_ref_col = [col for col in final_unweighted_table.columns if col.startswith("Referent")][0]

# Merge the two tables
final_merged_table = final_unweighted_table.merge(
    final_weighted_table,
    on=['Variable', 'Type'],
    how='outer'
)

# Reorder columns using the dynamic column names
final_merged_table = final_merged_table[[
    'Variable',
    'Type',
    unweighted_exp_col,
    unweighted_ref_col,
    'SMD',
    weighted_exp_col,
    weighted_ref_col,
    'Weighted SMD'
]]

# Clean and clarify column headers (remove line break hacks)
final_merged_table = final_merged_table.rename(columns={
    weighted_exp_col: weighted_exp_col.replace("\n", " "),
    weighted_ref_col: weighted_ref_col.replace("\n", " ")
})

# Display final table
final_merged_table

,Variable,Type,Exposure (n = 8022),Referent (n = 3330),SMD,Weighted Exposure (n = 1342),Weighted Referent (n = 1342),Weighted SMD
0,"Colonoscopy, Sigmoidoscopy",Categorical,726 (9.1%),239 (7.2%),0.069,101 (7.5%),101 (7.5%),-0.000
1,ACEi / ARB,Categorical,5867 (73.1%),2546 (76.5%),-0.077,1010 (75.3%),1010 (75.3%),0.000
2,ARNI,Categorical,582 (7.3%),126 (3.8%),0.152,69 (5.1%),69 (5.1%),-0.000
3,Acute kidney injury,Categorical,1482 (18.5%),677 (20.3%),-0.047,265 (19.7%),265 (19.7%),-0.000
4,Age,Continuous,"67.67 ± 9.19 [69, 62-74]","71.98 ± 8.55 [73, 67-78]",-0.485,"70.72 ± 8.41 [71, 66-76]","70.72 ± 8.81 [71, 66-77]",-0.000
...,...,...,...,...,...,...,...,...
155,Weight Categorization Score = Class 2 Obesity ...,Categorical,518 (6.5%),264 (7.9%),-0.057,105 (7.8%),105 (7.8%),-0.000
156,Weight Categorization Score = Class 3 Obesity ...,Categorical,4450 (55.5%),1261 (37.9%),0.358,600 (44.7%),600 (44.7%),-0.000
157,Weight Categorization Score = Unspecified Obesity,Categorical,2487 (31.0%),1249 (37.5%),-0.137,481 (35.8%),481 (35.8%),0.000
158,eGFR (Truncated),Continuous,"64.48 ± 23.53 [64, 46-83]","60.00 ± 22.37 [59, 44-76]",0.195,"62.43 ± 22.22 [61, 46-80]","60.92 ± 22.67 [59, 45-78]",0.068


In [11]:
# Your provided custom grouping
custom_grouping = {
    "Demographics": [
        "Age", "Gender (male)", "Race (Recategorized) = White", "Race (Recategorized) = Black", "Race (Recategorized) = Other", "Race (Recategorized) = Unknown / Missing",
        "Region / State = Northest", "Region / State = West", "Region / State = Midwest / North central", "Region / State = South", "Region / State = Missing"
    ],
    "Lifestyle risk factors": [
        "Smoking / Tobacco use", "Weight Categorization Score = Class 1 Obesity (30.0-34.9)", 
        "Weight Categorization Score = Class 2 Obesity (35.0-39.9)", "Weight Categorization Score = Class 3 Obesity (40.0 and above)", "Weight Categorization Score = Unspecified Obesity"
    ],
    "Diabetes complications": [
        "Diabetic retinopathy", "Diabetic neuropathy", "Diabetic nephropathy",
        "Diabetes with other opthalmic complications", "Diabetes with peripheral circulatory disorders",
        "Diabetic foot", "Erectile dysfunction", "Hypoglycemia", "Hyperglycemia / DKA / HONK", "Skin infections"
    ],
    "Cardiovascular-related conditions": [
        "Angina", "Atrial fibrillation", "Hypertension", "Hypotension",
        "Hyperlipidemia", "MI", "Ischemic stroke", "TIA",
        "Coronary atherosclerosis", "Cerebrovascular procedure",
        "Cardiac conduction disorder", "Other cardiac dysrhythmia", "Cardiomyopathy",
        "Valve disorders", "Edema", "Venous thromboembolism",
        "Pulmonary hypertension", "Hyperkalemia", "Insertion of pacemakers / removal of cardiac lead",
        "Implantable cardioverter defibrillator", "Previous cardiac procedure (CABG, PTCA, Stent)",
        "PVD diagnosis or surgery"
    ],
    "Renal-related conditions": [
        "CKD stage 1-2", "CKD stage 3-4", "Unspecified CKD", "Acute kidney injury", "Urolithiasis (kidney and urinary stone)",
        "Hypertensive nephropathy", "Urinary tract infections", "Genital infections"
    ],
    "Other comorbidities": [
        "COPD", "Asthma", "Obstructive sleep apnea", "Pneumonia", "Liver disease",
        "Hyperthyroidism and other thyroid gland disorders", "Hypothyroidism",
        "Fractures / Falls", "Osteoporosis", "Depression", "Dementia",
        "Delirium or psychosis", "Anemia", "Influenza", "Serious bacterial infections", "MASH/MASLD", 
        "Urinary incontinence", "Biliary disease", "Pancreatitis", "Bowel obstruction", "Gastroparesis"
    ],
    "Diabetes medications": [
        "Metformin", "Insulins", "Sulfonylureas", "DPP4i except sitagliptin",
        "SGLT2i", "Any other glucose-lowering drugs"
    ],
    "Heart failure medications": [
        "ACEi / ARB", "ARNI", "Thiazides", "Beta-blockers", "Calcium channel blockers",
        "Digoxin  / Digitoxin", "Loop diuretics", "Other diuretics",
        "Intravenous diuretics", "Nitrates"
    ],
    "Other medications": [
        "Anti-arrhythmics", "Statins", "PCSK9 inhibitors and other lipid-lowering drugs",
        "Antiplatelet agents", "Oral anticoagulants", "NSAIDs", "Oral corticosteroids",
        "Osteporosis agents (incl. bisphosphonates)", "Opioids", "Anti-depressants",
        "Antipsychotics", "Anxiolytics / hypnotics, benzos", "COPD / Asthma medications",
        "Urinary tract infections antibiotics", "Laxatives"
    ],
    "Healthcare utilization markers": [
        "Number of distinct medications", "Number of emergency department visits", "Number of office visits",
        "Number of endocrinologist visits ", "Number of cardiologist visits",
        "Number of internal/family medicine visits", "Electrocardiograms", "Echocardiograms",
        "Out-of-pocket medication cost", "Unique brand medications", "Unique generic medications",
        "Ratio of brand to generic medications"
    ],
    "Healthy behavior markers": [
        " Colonoscopy, Sigmoidoscopy", "Flu vaccine / Pneumococcal vaccine", "Pap smear",
        "PSA test", "Fecal occult blood test", "Bone mineral density tests", "Mammograms"
    ],
    "Laboratory and diagnostic tests": [
        "HbA1c tests", "Lipid panel", "Creatinine tests", "Urine tests"
    ],
    "Lab values": [
        "BMI (Truncated)", "BNP (Truncated)", "proBNP (Truncated)", "HbA1c (Truncated)",
        "Glucose (Truncated)", "eGFR (Truncated)", "Creatinine (Truncated)",
        "Systolic blood pressure (Truncated)", "Heart rate (Truncated)"
    ],
    "Burden of comorbidities": [
        "Combined comorbidity score (CCI), 365 days", "Frailty Score: Empirical Version 365 days (ICD-9/10)"
    ],
    "Baseline hospitalizations and hospital metrics": [
        "Number of any hospitalization", "Number of heart failure hospitalizations (0, 1, 2 or more) = 0", "Number of heart failure hospitalizations (0, 1, 2 or more) = 1", "Number of heart failure hospitalizations (0, 1, 2 or more) = 2 or more",
        "Number of hospitalizations (0, 1, 2 or more) = 0", "Number of hospitalizations (0, 1, 2 or more) = 1", "Number of hospitalizations (0, 1, 2 or more) = 2 or more"
    ],
    "Calendar year of cohort entry": [
        "Entry Classification = 2018", "Entry Classification = 2019", "Entry Classification = 2020", "Entry Classification = 2021", "Entry Classification = 2022", "Entry Classification = 2023", "Entry Classification = 2024"
    ]
}

# Build final ordered table
ordered_rows = []
for group, variables in custom_grouping.items():
    ordered_rows.append(pd.Series({"Variable": group}))  # Heading
    group_rows = final_merged_table[final_merged_table['Variable'].isin(variables)]
    ordered_rows.extend([row for _, row in group_rows.iterrows()])

# Create the final table
final_ordered_table = pd.DataFrame(ordered_rows).reset_index(drop=True)

# Display
final_ordered_table

,Variable,Type,Exposure (n = 8022),Referent (n = 3330),SMD,Weighted Exposure (n = 1342),Weighted Referent (n = 1342),Weighted SMD
0,Demographics,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Age,Continuous,"67.67 ± 9.19 [69, 62-74]","71.98 ± 8.55 [73, 67-78]",-0.485,"70.72 ± 8.41 [71, 66-76]","70.72 ± 8.81 [71, 66-77]",-0.000
2,Gender (male),Categorical,3594 (44.8%),1463 (43.9%),0.017,597 (44.5%),597 (44.5%),0.000
3,Race (Recategorized) = Black,Categorical,1494 (18.6%),663 (19.9%),-0.033,260 (19.4%),260 (19.4%),-0.000
4,Race (Recategorized) = Other,Categorical,64 (0.8%),36 (1.1%),-0.029,13 (0.9%),13 (0.9%),-0.000
...,...,...,...,...,...,...,...,...
167,Entry Classification = 2020,Categorical,442 (5.5%),605 (18.2%),-0.400,203 (15.1%),203 (15.1%),-0.000
168,Entry Classification = 2021,Categorical,683 (8.5%),476 (14.3%),-0.183,221 (16.5%),221 (16.5%),-0.000
169,Entry Classification = 2022,Categorical,1206 (15.0%),402 (12.1%),0.087,244 (18.2%),244 (18.2%),-0.000
170,Entry Classification = 2023,Categorical,2542 (31.7%),355 (10.7%),0.533,277 (20.6%),277 (20.6%),-0.000


In [ ]:
# Save final dataset
merged_data.to_csv("merged_overlap_weights.csv", index=False)

# Optional: Plot distribution of weights
import matplotlib.pyplot as plt
merged_data['overlap_weight'].hist(bins=50)
plt.title("Distribution of Overlap Weights")
plt.xlabel("Weight")
plt.ylabel("Count")
plt.show()